In [2]:
import numpy as np
import pandas as pd

import arviz as az

import bebi103

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

/home/ec2-user/miniconda/lib/python3.7/site-packages/bebi103/viz.py:30: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  Features requiring DataShader will not work and you will get exceptions.""")


Loading BokehJS ...

## Problem 8.2: Microtubule catastrophe, 40 pts

_Note: This problem is best done after the lecture November 22._

In this problem, we use data from [Gardner, Zanic, et al., Depolymerizing kinesins Kip3 and MCAK shape cellular microtubule architecture by differential control of catastrophe, *Cell*, **147**, 1092-1103, 2011](https://doi.org/10.1016/j.cell.2011.10.037). The authors investigated the dynamics of microtubule catastrophe, the switching of a microtubule from a growing to a shrinking state.  In particular, they were interested in the time between the start of growth of a microtubule and the catastrophe event. They monitored microtubules in a single-molecule [TIRF assay](https://en.wikipedia.org/wiki/Total_internal_reflection_fluorescence_microscope) by using tubulin (the monomer that comprises a microtubule) that was labeled with a fluorescent marker. As a control to make sure that fluorescent labels and exposure to laser light did not affect the microtubule dynamics, they performed a similar experiment using differential interference contrast (DIC) microscopy. They measured the time until catastrophe with labeled and unlabeled tubulin. We will carefully analyze the data and make some conclusions about the processes underlying microtubule catastrophe.

In the file `gardner_mt_catastrophe_only_tubulin.csv` (which you can download [here](../data/gardner_mt_catastrophe_only_tubulin.csv)), we have observed catastrophe times of microtubules with different concentrations of tubulin. To start with, we will consider the experiment run with a tubulin concentration of 12 µM. So, our data set consists of a set of measurements of the amount of time to catastrophe. We will consider three models for microtubule catastrophe.

- Model 1: The time to catastrophe is Exponentially distributed.
- Model 2: The time to catastrophe is Gamma distributed.
- Model 3: The time to catastrophe is Weibull distributed.

Note that these descriptions are for the likelihood; we have not specified priors.


**a)  Describe the three models in words. Give physical descriptions of the meanings of their parameters. Describe how these models are related to each other. Tutorial 3c will be useful.** 


- Model 1: The time to catastrophe is Exponentially distributed.

This suggests that the occurance of catastrophe is a Poisson process, so it is a "rare event" that requires multiple subprocesses to lead it it. The parameter for the process, if it is exponential, $\beta$, represents the characteristic rate of catastrophe, that is how often catastrophe happens in a certain amount of time. It can also be parametrized as $\tau=1/\beta$, the characteristic catastrophe time, which fits what we are given in our data. The Exponential distribution is a special case of the Gamma distribution where $\alpha = 1$ and a special case of the Weibull distribution where $\alpha = 1$ and $\sigma=1/\beta$



- Model 2: The time to catastrophe is Gamma distributed.

This suggests that the occurance of catastrophe represents a specific number of occurances of a Poisson process, that is a discrete number of steps that occur at the same rate must occur for catastrophe to occur. There are two parameters for this distribution, $\alpha$ and $\beta$, where $\alpha$ is the number of arrivals (or "steps") required to trigger catastrophe, and $\beta$ is the rate of the arrivals. Thus, the characteristic catastrophe time is given by $\alpha/\beta$. 


- Model 3: The time to catastrophe is Weibull distributed.

This suggests that the likelihood of catastrophe is dependent on the amount of time it has been since the last catastrophe, so the longer it has been since the last catastrophe, the more likely it is that catastrophe will occur. There are two parameters for this distribution, $\alpha$ which defines how the probability changes over time, and $\sigma$ which is the characteristic catastrophe time.

**b) Perform parameter estimates for the respective models and make model comparisons. Comment on what this means with respect to our understanding of how microtubule catastrophe works.**

We first load in our data and come up with our priors for the parameters of the three distributions. We don't have much prior knowledge so we will keep them simple (normal distribution).

**Exponential(beta)**

tao ~ normal(400, 100)

beta = 1/tao

We guess that the castastrophe times will be in the hundreds.

**Gamma($\alpha$, $\beta$)**

alpha ~ normal(10, 3)

We assume that it will take around 10 arrivals to trigger catastrophe.

tao ~ normal(150, 50)

beta = 1/tao


**Weibull($\alpha$, $\sigma$)**

$\alpha$ ~ normal(1, 0.1)

We think that the rate of catastrophe is not dependent on the time elapsed from the last event, so we center our distribution aroud 1 without much variation.

$\sigma$ ~ normal(10, 3)


In [3]:
df = pd.read_csv('/home/ec2-user/data/gardner_mt_catastrophe_only_tubulin.csv', comment = "#")

In [4]:
# Check out the data we are working with
df.head()

,12 uM,7 uM,9 uM,10 uM,14 uM
0,25.000,35.0,25.0,50.0,60.0
1,40.000,45.0,40.0,60.0,75.0
2,40.000,50.0,40.0,60.0,75.0
3,45.429,50.0,45.0,75.0,85.0
4,50.000,55.0,50.0,75.0,115.0


In [15]:
prior_sm1 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m1.stan')
prior_sm2 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m2.stan')
prior_sm3 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m3.stan')

Using cached StanModel.
Using cached StanModel.
Using cached StanModel.


In [7]:
# Store input parameters in a dictionary so stan can access them
data = dict(N=692)

# Generate samples
samples_gen1 = prior_sm1.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)
samples_gen2 = prior_sm2.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)
samples_gen3 = prior_sm3.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)

# Store samples in a dataframe
df_gen1 = bebi103.stan.to_dataframe(samples_gen1, diagnostics=False)
df_gen2 = bebi103.stan.to_dataframe(samples_gen2, diagnostics=False)
df_gen3 = bebi103.stan.to_dataframe(samples_gen3, diagnostics=False)

# Let's look at one of the dataframes to make sure they look ok
df_gen3.head()

,chain,chain_idx,warmup,time[1],time[2],time[3],time[4],time[5],time[6],time[7],...,time[686],time[687],time[688],time[689],time[690],time[691],time[692],alpha,sigma,lp__
0,1,1,0,1102.172664,794.759346,322.784161,6.998034,309.412229,58.203859,243.011888,...,515.824878,523.972442,560.689080,328.276135,916.134372,171.260329,89.805550,0.940352,379.604562,0.0
1,1,2,0,15.573187,458.291165,135.568627,485.515067,636.306628,70.104197,26.076996,...,3.910033,404.223505,85.720805,90.430855,127.677189,46.651275,72.403660,0.910429,286.436602,0.0
2,1,3,0,205.616295,16.502412,1212.666061,751.336065,177.386971,6.681908,936.959174,...,250.526643,12.341789,95.151978,500.540105,225.821139,95.337844,1361.980940,1.075877,410.672061,0.0
3,1,4,0,1142.666242,104.619770,929.346283,303.019985,85.682187,345.015274,213.638790,...,9.809741,1020.808087,316.887235,128.310563,1070.668953,630.908613,388.045371,0.865075,474.237019,0.0
4,1,5,0,107.339672,1103.720935,185.799321,355.669812,624.239307,708.899676,161.795091,...,27.311829,649.744322,44.862176,22.137954,522.369859,243.324547,990.477375,1.066718,608.000472,0.0


In [8]:
p = bebi103.viz.predictive_ecdf(samples_gen1, "time",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 6000)
bokeh.io.show(p)

In [9]:
p = bebi103.viz.predictive_ecdf(samples_gen2, "time",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 3000)
bokeh.io.show(p)

In [10]:
p = bebi103.viz.predictive_ecdf(samples_gen3, "time",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 1000)
bokeh.io.show(p)

Our priors look reasonable, so now we can move on to creating our mcmc models and sampling.

In [11]:
sm1 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m1.stan')
sm2 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m2.stan')
sm3 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m3.stan')

Using cached StanModel.
Using cached StanModel.
Using cached StanModel.


In [16]:
data = dict(N=len(df),
           time=df['12 uM'].values.astype(float))

In [17]:
samples1 = sm1.sampling(data=data)
samples2 = sm2.sampling(data=data)
samples3 = sm3.sampling(data=data)

In [18]:
df_mcmc1 = bebi103.stan.to_dataframe(samples1, diagnostics=False, inc_warmup=False)
df_mcmc2 = bebi103.stan.to_dataframe(samples2, diagnostics=False, inc_warmup=False)
df_mcmc3 = bebi103.stan.to_dataframe(samples3, diagnostics=False, inc_warmup=False)

In [103]:
df_mcmc2.head()

,chain,chain_idx,warmup,alpha_,tao,beta_,log_like[1],log_like[2],log_like[3],log_like[4],...,uM_12_ppc[684],uM_12_ppc[685],uM_12_ppc[686],uM_12_ppc[687],uM_12_ppc[688],uM_12_ppc[689],uM_12_ppc[690],uM_12_ppc[691],uM_12_ppc[692],lp__
0,1,1,0,3.209207,120.434039,0.008303,-9.366675,-8.452889,-8.452889,-8.216800,...,442.920838,130.596781,544.982739,355.310202,611.936148,463.786329,273.267714,202.050533,291.451031,-4665.081082
1,1,2,0,3.282229,115.122574,0.008686,-9.417162,-8.474803,-8.474803,-8.231499,...,104.846017,672.417164,674.571295,487.266927,355.341179,387.371020,226.663029,390.530349,356.099498,-4664.955822
2,1,3,0,3.254664,115.634370,0.008648,-9.360329,-8.430348,-8.430348,-8.190344,...,633.464358,361.045293,201.137435,131.587049,43.832017,131.163949,237.005530,239.270318,271.472878,-4664.869484
3,1,4,0,3.032317,125.441544,0.007972,-9.032365,-8.196746,-8.196746,-7.981369,...,97.901472,340.156061,231.495826,915.474799,567.587235,280.079977,338.736988,318.629583,438.976129,-4664.912656
4,1,5,0,3.296510,111.339886,0.008982,-9.350940,-8.406294,-8.406294,-8.162775,...,298.654358,352.065525,484.729064,312.848749,1242.585268,286.294185,487.338594,367.568810,230.652195,-4666.260687


We now plot the post predictive checks to do model comparison.

In [21]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples1, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

We can see there is a systematic issue with using the exponential distribution becuase the two inflection points will always be in the wrong place.

In [22]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples2, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

The gamma distribution looks a lot more promising! Most of our values fall into that innermost confidence interval.

In [23]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples3, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

Ooof the weibull distribution model is also not great (better than exponential though). Only a few of our datapoints fall in that 99% confidence interval.

To confirm quantitatively, we compute the loo given the log likelihood and use the bebi103 compare function to calculate the loo and the weights.

In [24]:
bebi103.stan.compare({'exponential': samples1, 'gamma': samples2, 'weibull': samples3},
                     log_likelihood='log_like', ic='loo')

,loo,ploo,dloo,weight,se,dse,warning
gamma,9278.55,2.21662,0,0.988585,45.4097,0,0
weibull,9321.64,2.19449,43.0974,0,43.0359,9.46304,0
exponential,9608.59,0.367672,330.043,0.0114146,31.8857,28.3334,0


We can see that gamma (Model 2) is the best distribution to fit our data because the weight is much higher (0.95 vs 10^-2 and 10^-13). Based on the visual analysis of the post predictive checks above,  this makes a lot of sense! 

**c) Using whichever model you favor based on your work in part (b), obtain parameter estimates for the other tubulin concentrations. Given that microtubules polymerize faster with higher tubulin concentrations, is there anything you can say about the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?**


We picked the gamma distribution model so we will now sample out of it for all 5 concentrations in the dataframe.

In [27]:
data12 = dict(N=len(df),
           time=df['12 uM'].values.astype(float))
data7 = dict(N=len(df['7 uM'].dropna()),
           time=df['7 uM'].dropna().values.astype(float))
data9 = dict(N=len(df['9 uM'].dropna()),
           time=df['9 uM'].dropna().values.astype(float))
data10 = dict(N=len(df['10 uM'].dropna()),
           time=df['10 uM'].dropna().values.astype(float))
data14 = dict(N=len(df['14 uM'].dropna()),
           time=df['14 uM'].dropna().values.astype(float))

In [28]:
samples12 = sm2.sampling(data=data12)
samples7 = sm2.sampling(data=data7)
samples9 = sm2.sampling(data=data9)
samples10 = sm2.sampling(data=data10)
samples14 = sm2.sampling(data=data14)

We now preform post predictive checks and can conlcude that the gamma model is resoanble for all 5 concentrations.

In [33]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples12, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False,
                                          title = "12 uM",
                                          x_axis_label = "intercatastrophe time (s)"))

In [31]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples7, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['7 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "7 uM",
                                          x_axis_label = "intercatastrophe time (s)"))

In [34]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples9, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['9 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "9 uM",
                                          x_axis_label = "intercatastrophe time (s)"))

In [36]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples10, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['10 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "10 uM",
                                          x_axis_label = "intercatastrophe time (s)"))

In [37]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples14, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['14 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "14 uM",
                                          x_axis_label = "intercatastrophe time (s)"))

Now we want to examine the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?

There are two parameters for this distribution,  𝛼  and  𝛽 , where  𝛼  is the number of arrivals (or "steps") required to trigger catastrophe, and  𝛽  is the rate of the arrivals. Thus, the characteristic catastrophe time is given by  𝛼/𝛽 .

We first need to put our sampled values in a dataframe so we can look at the parameters

In [29]:
df_mcmc12 = bebi103.stan.to_dataframe(samples12, diagnostics=False, inc_warmup=False)
df_mcmc7 = bebi103.stan.to_dataframe(samples7, diagnostics=False, inc_warmup=False)
df_mcmc9 = bebi103.stan.to_dataframe(samples9, diagnostics=False, inc_warmup=False)
df_mcmc10 = bebi103.stan.to_dataframe(samples10, diagnostics=False, inc_warmup=False)
df_mcmc14 = bebi103.stan.to_dataframe(samples14, diagnostics=False, inc_warmup=False)

Then, we can calculate the average characteristic catastrophe time (given by  𝛼/𝛽 ) for each concentration.

In [45]:
ab_7 = df_mcmc7["alpha_"].mean() / df_mcmc7["beta_"].mean()
ab_9 = df_mcmc9["alpha_"].mean() / df_mcmc9["beta_"].mean()
ab_10 = df_mcmc10["alpha_"].mean() / df_mcmc10["beta_"].mean()
ab_12 = df_mcmc12["alpha_"].mean() / df_mcmc12["beta_"].mean()
ab_14 = df_mcmc14["alpha_"].mean() / df_mcmc14["beta_"].mean()

We print them out in order of increasing concentration to try to see a trend.

In [48]:
{"7" : ab_7, "9" : ab_9, "10" : ab_10, "12" : ab_12, "14" : ab_14}

{'7': 324.0935908265683,
 '9': 306.3749224970161,
 '10': 356.8564577103455,
 '12': 380.84820105239703,
 '14': 469.654885839185}

As concentration increases, the characteristic catastrophe time seems to generally increase (the 9uM data breaks this trend). This makes sense given that microtubules polymerize faster with higher tubulin concentrations.